## Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

1) We are a fast food brand looking for the best place to open a shop in Toronto. In general we are looking for a place of concentration families with kids, because children are just crazy about cheap and tasty food and don't care if it is healthy or not.

2) Probably, that place already has other fast foods nearby (we are not the only smart guys here), this will be the first marker. A bakery or confectionery would be another sign of a correct place for us. And the third marker could be an event place or a playgound. Idially, there won't be many competitiors to our business. Let's analyze the neighbourghoods and see if there is something suitable for us.

## Data <a name="data"></a>

1) Get postal codes data  from Wiki (within the city of Toronto in the province of Ontario)

In [84]:
from bs4 import BeautifulSoup
import requests
import csv
import lxml
import re
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [85]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

my_table = soup.find('table',{'class':'wikitable sortable'})
thread = my_table.findAll('th')
thread = str(thread).split(',')
for el in range(len(thread)):
    thread[el] = thread[el].replace("[", "")
    thread[el] = thread[el].replace("]", "")
    thread[el] = thread[el].replace("<th>", "")
    thread[el] = thread[el].replace("</th>", "")    
    thread[el] = thread[el].replace("\n", "") 
    thread[el] = thread[el].strip() 
    
tbody = my_table.findAll('td')
#print(tbody)
table = []
for el in range(0, len(tbody), 3):
    temp_list = []
    temp_list.append(tbody[el])    
    temp_list.append(tbody[el+1])
    temp_list.append(tbody[el+2])
    table.append(temp_list)
#print(table)
def pattern_recognition(word):
    code1 = re.compile('title=\"(.*?)\"')
    match_code1 = code1.search(word)
    if match_code1:
        return match_code1.group(0)
    
    
    
frame = []
for unit in table:
    temp_frame = []
    for el in unit:
        el = str(el).replace("<td>", "")
        el = str(el).replace("</td>", "")    
        el = str(el).replace("\n", "")
        pattern = pattern_recognition(el)
        if pattern != None:
            pattern = pattern.replace("title=\"", "")
            pattern = pattern.replace("\"", "")                          
            temp_frame.append(pattern)
        else:
            temp_frame.append(el)
    frame.append(temp_frame) 

df = pd.DataFrame(frame)
df.columns = [thread[0], thread[1], thread[2]]
df.replace('Not assigned', np.NaN, inplace=True)
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),NaN
9,M8A,NaN,NaN


In [86]:
df.shape

(288, 3)

In [87]:
df.Borough.unique()

array([nan, 'North York', 'Downtown Toronto', "Queen's Park (Toronto)",
       'Etobicoke', 'Scarborough, Toronto', 'East York', 'York',
       'East Toronto', 'West Toronto', 'Central Toronto', 'Mississauga'],
      dtype=object)

In [88]:
df['Neighbourhood'].isnull().value_counts()

False    210
True      78
Name: Neighbourhood, dtype: int64

In [89]:
df['Borough'].isnull().value_counts()

False    211
True      77
Name: Borough, dtype: int64

In [90]:
df = df[df['Borough'].notnull()]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [91]:
df.shape

(211, 3)

In [92]:
df['Neighbourhood'] = df.apply(
    lambda row: row['Borough'] if pd.isnull(row['Neighbourhood']) else row['Neighbourhood'],
    axis=1)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),Queen's Park (Toronto)
10,M9A,Etobicoke,Islington Avenue
11,M1B,"Scarborough, Toronto","Rouge, Toronto"
12,M1B,"Scarborough, Toronto","Malvern, Toronto"


In [93]:
df.index = pd.RangeIndex(len(df.index))
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront (Toronto)
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [94]:
result1 = [df['Postcode'][0]]
result2 = [df['Borough'][0]]
result3 = [df['Neighbourhood'][0]]

for k, a, b in zip(df['Postcode'][1:], df['Borough'][1:], df['Neighbourhood'][1:]):
    if k == result1[-1] and b != result3[-1]:        # If k matches the last value in result1,
        result3[-1] += ", " + b  # add a to the last value of result3
    else:  # Otherwise add a new row with the values
        result1.append(k)
        result2.append(a)
        result3.append(b)

# Create a new dataframe using these result lists
data = pd.DataFrame({'Postcode': result1, 'Borough': result2, 'Neighbourhood': result3})
data.head(20)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park (Toronto)
5,M9A,Etobicoke,Islington Avenue
6,M1B,"Scarborough, Toronto","Rouge, Toronto, Malvern, Toronto"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [95]:
data.shape

(103, 3)

2) Add GPS coordinates

In [96]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [97]:
df_coord.shape

(103, 3)

In [98]:
df_coord.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [121]:
frames = [data,df_coord]
df_keys = pd.merge(data,df_coord, on='Postcode')
df_keys.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Queen's Park (Toronto),43.662301,-79.389494


In [122]:
df_keys.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df_keys.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Queen's Park (Toronto),43.662301,-79.389494


## Methodology  <a name="data"></a>

In this project we will be looking for the best places to open a new fast food restaurant.
The places must satisfy the following criterias:
- suitable for families with children
- presence of bakery or confectionery
- presence of event place or playground
To do that we need to get information about venue already existed in each area using Foursquare API and pull out the most common ones. Then cluster all the areas using k-mean classifier. And analyse whether a certain cluster match our criterias or not.

## Analysis  <a name="data"></a>

Let's perform some explanatory data analysis and derive some additional info from our raw data

1) Plotting the data

In [123]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from geopy.geocoders import Nominatim

In [124]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [125]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_keys['Latitude'], df_keys['Longitude'], df_keys['Borough'], df_keys['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

2) Get venue data in each Neighbourhood using Foursquare

In [126]:
CLIENT_ID = 'OSUQFPQS1YKMG4YH1AN5BHBHPKGTNAEHSPXYDUR5URYXEHTX'
CLIENT_SECRET = '1U0U0DMU4CL4AQRLILOSB4VVVFFZYIR1VMNUBY5CEPEMRE1Z'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OSUQFPQS1YKMG4YH1AN5BHBHPKGTNAEHSPXYDUR5URYXEHTX
CLIENT_SECRET:1U0U0DMU4CL4AQRLILOSB4VVVFFZYIR1VMNUBY5CEPEMRE1Z


3) Explore Neighborhoods

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
Toronto_venues = getNearbyVenues(names=df_keys['Neighbourhood'],
                                   latitudes=df_keys['Latitude'],
                                   longitudes=df_keys['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront (Toronto), Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park (Toronto)
Islington Avenue
Rouge, Toronto, Malvern, Toronto
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Toronto, Martin Grove, Princess Gardens, West Deane Park
Highland Creek (Toronto), Rouge Hill, Port Union, Toronto
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, Toronto, West Hill, Toronto
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn, Toronto
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights, Toronto
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station (Toronto)
Little

In [127]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2247, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [128]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Ontario, Steeles East",2,2,2,2,2,2
"Agincourt, Toronto",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Mount Olive-Silverstone-Jamestown, Mount Olive-Silverstone-Jamestown, Silverstone, Toronto, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Toronto, Long Branch, Toronto",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights, Toronto",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Toronto, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55


In [129]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


4) Analyze Each Neighborhood in Toronto

In [130]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.shape

(100, 276)

5) Create a pandas dataframe with each neighborhood and it top 5 most common venues

In [132]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Ontario, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----Agincourt, Toronto----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3      Clothing Store  0.25
4  Miscellaneous Shop  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Mount Olive-Silverstone-Jamestown, Mount Olive-Silverstone-Jamestown, Silverstone, Toronto, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.17
1          Pizza Place  0.08
2  Japanese Restaurant  0.08
3             Pharmacy  0.08
4           Beer Store  0.08


----Alderwood, Toro

              venue  freq
0    Discount Store  0.29
1  Department Store  0.14
2        Hobby Shop  0.14
3       Coffee Shop  0.14
4       Bus Station  0.14


----East Toronto----
                       venue  freq
0                       Park  0.50
1                Coffee Shop  0.25
2          Convenience Store  0.25
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00


----Emery, Toronto, Humberlea----
                             venue  freq
0           Furniture / Home Store   0.5
1                   Baseball Field   0.5
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.15
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3            Restaurant  0.03
4           Bus Station  0.03


----First Canadian Place, Underground city----
                 venue  freq
0          Coffee Shop  0.08
1  

                             venue  freq
0                       Playground   1.0
1                Accessories Store   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Silver Hills, York Mills----
                       venue  freq
0                  Cafeteria   0.5
1          Martial Arts Dojo   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2            Café  0.05
3           Hotel  0.05
4  Breakfast Spot  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.04
2          Restaurant  0.04
3  Seafood Restaurant  0.03
4               Hotel  0.03


----Studio District----
                venue  freq
0                Café  0.11
1         Coffee Shop  0.08
2              Baker

In [139]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,American Restaurant,Restaurant,Burger Joint,Sushi Restaurant,Gym
1,"Agincourt North, L'Amoreaux East, Milliken, On...",Park,Playground,Yoga Studio,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt, Toronto",Lounge,Sandwich Place,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Japanese Restaurant,Fast Food Restaurant,Liquor Store,Discount Store,Pharmacy,Pizza Place,Sandwich Place
4,"Alderwood, Toronto, Long Branch, Toronto",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Dance Studio,Sandwich Place,Coffee Shop,General Entertainment


6) Cluster Neighborhoods into 5 similar groups

In [141]:
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_keys
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
Toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Bus Stop,Park,Fast Food Restaurant,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Italian Restaurant,Chocolate Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Miscellaneous Shop,Shoe Store,Coffee Shop,Vietnamese Restaurant
4,M7A,Queen's Park (Toronto),Queen's Park (Toronto),43.662301,-79.389494,0.0,Coffee Shop,Gym,Japanese Restaurant,Diner,Burger Joint,Yoga Studio,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burrito Place


In [142]:
Toronto_merged['Cluster Labels'].value_counts()

0.0    83
2.0    13
1.0     2
3.0     1
4.0     1
Name: Cluster Labels, dtype: int64

In [143]:
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].fillna(0)
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)
Toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Bus Stop,Park,Fast Food Restaurant,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Italian Restaurant,Chocolate Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Miscellaneous Shop,Shoe Store,Coffee Shop,Vietnamese Restaurant
4,M7A,Queen's Park (Toronto),Queen's Park (Toronto),43.662301,-79.389494,0,Coffee Shop,Gym,Japanese Restaurant,Diner,Burger Joint,Yoga Studio,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burrito Place


In [144]:
Toronto_merged['Cluster Labels'].unique()

array([2, 0, 4, 1, 3], dtype=int64)

7) Plot the clustered data

In [148]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0:

In [149]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Italian Restaurant,Chocolate Shop
3,North York,0,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Miscellaneous Shop,Shoe Store,Coffee Shop,Vietnamese Restaurant
4,Queen's Park (Toronto),0,Coffee Shop,Gym,Japanese Restaurant,Diner,Burger Joint,Yoga Studio,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burrito Place
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Scarborough, Toronto",0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant
7,North York,0,Gym / Fitness Center,Baseball Field,Café,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,East York,0,Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Pharmacy,Café,Rock Climbing Spot,Bank
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Diner,Restaurant,Lingerie Store,Ramen Restaurant
12,"Scarborough, Toronto",0,Bar,History Museum,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


#### Cluster 1 :

In [150]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,"Scarborough, Toronto",1,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
85,"Scarborough, Toronto",1,Park,Playground,Yoga Studio,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


#### Cluster 2 :

In [151]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Bus Stop,Park,Fast Food Restaurant,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
10,North York,2,Park,Japanese Restaurant,Bakery,Pub,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
21,York,2,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
35,East York,2,Park,Convenience Store,Coffee Shop,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
40,North York,2,Park,Bus Stop,Airport,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
49,North York,2,Basketball Court,Park,Bakery,Construction & Landscaping,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
61,Central Toronto,2,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
64,York,2,Park,Convenience Store,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
66,North York,2,Park,Bank,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
68,Central Toronto,2,Park,Trail,Sushi Restaurant,Jewelry Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega


#### Cluster 3 :

In [152]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Garden,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Deli / Bodega


#### Cluster 4:

In [153]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,4,Bank,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store


## Results and Discussion  <a name="data"></a>

The best of all clusters above is Cluster 1 for the following reasons:
- Location. It's located aside of downtown, but no too far. Which is a good place for families with children.
- The key markers matched. The area has Playground, Donut Shop, Dessert Shop.
- No other fast foods in around. The competition will be less.
- Some Boroughs in Cluster 0 would satisfy our criterias, but competition would be higher there.
- Other clusters don't match the requirements at all.

## Conclusion  <a name="data"></a>

The goal of the project was fulfilled. The best borough for our goal is Scarborough, Toronto.